In [1]:
from lmfit import minimize
from lmfit import Parameters

In [2]:
import pyroll.core as pr
import pyroll.pillar_model
import pyroll.neutral_point_estimator
import pyroll.sparling_spreading
import pyroll.hensel_force_torque
import pyroll.freiberg_flow_stress
import pyroll.ring_model_thermal

In [3]:
from in_profile import create_in_profile_from_surface_temperature
from rolling_sequence import roughing_sequence, intermediate_sequence, finishing_sequence_e112, finishing_sequence_e107
from utilities import *

In [4]:
pyroll.pillar_model.Config.PILLAR_COUNT = 30
pyroll.neutral_point_estimator.Config.ESTIMATOR = "EQUAL"
ROLL_SURFACE_TEMPERATURE = 50 + 273.15
DISK_ELEMENT_COUNT = 30

In [5]:
@pr.CoolingPipe.DiskElement.heat_transfer_coefficient
def heat_transfer_coefficient_model_moreales(self: pr.CoolingPipe.DiskElement):
    mean_surface_temperature = (self.in_profile.surface_temperature + self.out_profile.surface_temperature) / 2
    profile_surface_temperature_celsius = mean_surface_temperature - 273.15
    return 13950 + 5.12 * 1e6 / profile_surface_temperature_celsius

In [6]:
in_profile = create_in_profile_from_surface_temperature(1050 + 273.15)
in_profile.ring_temperatures = in_profile.ring_temperatures

In [7]:
roughing_train = roughing_sequence(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

intermediate_train = intermediate_sequence(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

finishing_train_e112 = finishing_sequence_e112(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)

finishing_train_e107 = finishing_sequence_e107(
    roll_surface_temperature=ROLL_SURFACE_TEMPERATURE,
    disk_element_count=DISK_ELEMENT_COUNT
)


In [8]:
rolling_train_e112 = pr.PassSequence([
    roughing_train,
    intermediate_train,
    finishing_train_e112
])

rolling_train_e107 = pr.PassSequence([
    roughing_train,
    intermediate_train,
    finishing_train_e107
])

In [9]:
rolling_train_e112.flatten()
rolling_train_e107.flatten()

In [10]:
rolling_train_e112.solve(in_profile)

KeyboardInterrupt: 

In [ ]:
rolling_train_e107.solve(in_profile)

In [47]:
def solve_total_wear_depths_as_contour_line(params):
    sequences = [
        {
            "label": "e11_2",
            "sequence": rolling_train_e112
        },
        {
            "label": "e10_7",
            "sequence": rolling_train_e107
        }
    ]

    area_errors = []
    wear_coefficient = params['wear_coefficient'].value

    for dict in sequences:
        roll_passes = dict["sequence"].roll_passes
        for roll_pass in roll_passes:
            labels, tonnages, measured_wear_contours = extract_wear_contours_from_measurement(dict["label"])
        for label, tonnage, measured_wear_contour in zip(labels, tonnages, measured_wear_contours):
            if roll_pass.label == label:
                calculated_wear_contour = calculate_wear_contour(roll_pass, wear_coefficient, tonnage * 1000)
                area_error = calculate_area_between_contours(calculated_wear_contour, measured_wear_contour)

                area_errors.append(area_error)
            else:
                continue

    rms_error = root_mean_square_value(np.array(area_errors))
    print("RMS Error: ", rms_error)
    return rms_error


In [48]:
params = Parameters()
params.add('wear_coefficient', value=1e-6, min=1e-10, max=1e-3, vary=True)

In [49]:
result = minimize(
    solve_total_wear_depths_as_contour_line,
    params,
    nan_policy='omit',
    max_nfev=100,
    method="cobyla"
)

Area error:  0.016834346823023448
Area error:  0.010227192945194166
Area error:  0.04900690185349483
Area error:  0.010295584418315308
Area error:  0.023222075066337763
Area error:  0.008776070341056565
Area error:  0.016852583875867105
Area error:  0.00812307846281863
RMS Error:  0.02197141428172309
Area error:  4.3265347886895364
Area error:  2.6284648422998553
Area error:  12.595619744233055
Area error:  2.6458620808845095
Area error:  5.9684351017286925
Area error:  2.255606627101959
Area error:  4.331633694693945
Area error:  2.087701730519397
RMS Error:  5.647006744702435
Area error:  3.4310378167537587
Area error:  2.0844307688342063
Area error:  9.988604718489086
Area error:  2.098227309336746
Area error:  4.733100921382851
Area error:  1.7887458818663193
Area error:  3.435081029045135
Area error:  1.6555936499372543
RMS Error:  4.478201122739372
Area error:  0.7901683264158526
Area error:  0.48004456614951263
Area error:  2.300374988960189
Area error:  0.48322245130538105
Area

In [50]:
result.params

name,value,initial value,min,max,vary
wear_coefficient,1.5398e-10,1e-06,1.0000e-10,1.0000e-03,True
